In [1]:
print('Importing Start')
import os
import sys
print(sys.version)         # full version string
print(sys.version_info)    # tuple of (major, minor, micro, ...)
import pandas as pd
import numpy as np
import torch
import re
import ast
import json
import warnings
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import faiss
import logging
#import ospip
print('Adding directory')
# Add the directory containing the file to sys.path
module_path = os.path.abspath(os.path.join('..', '99_6.C01'))
if module_path not in sys.path:
    sys.path.append(module_path)
from load_complete_df import return_df_final
from RecipeEmbedder_new import RecipeEmbedder
from RecipePreprocessor import RecipePreprocessor
print('Importing Finished')
# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

Importing Start
3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:23:34) [Clang 18.1.8 ]
sys.version_info(major=3, minor=10, micro=17, releaselevel='final', serial=0)


/Users/maxknuth/opt/anaconda3/envs/recipe_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-30 00:54:55,013 - INFO - Using device: cpu


Adding directory
Importing Start
3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:23:34) [Clang 18.1.8 ]
sys.version_info(major=3, minor=10, micro=17, releaselevel='final', serial=0)
Importing Start
3.10.17 | packaged by conda-forge | (main, Apr 10 2025, 22:23:34) [Clang 18.1.8 ]
sys.version_info(major=3, minor=10, micro=17, releaselevel='final', serial=0)
Importing Finished


In [2]:
"""  # Load the dataset
logger.info("Loading dataset...")
data_raw = return_df_final()
logger.info(f"Dataset loaded with shape: {data_raw.shape}")

# Display basic column information
logger.info(f"Columns: {data_raw.columns.tolist()}")
data_raw.to_pickle('raw_data.pkl')"""


'  # Load the dataset\nlogger.info("Loading dataset...")\ndata_raw = return_df_final()\nlogger.info(f"Dataset loaded with shape: {data_raw.shape}")\n\n# Display basic column information\nlogger.info(f"Columns: {data_raw.columns.tolist()}")\ndata_raw.to_pickle(\'raw_data.pkl\')'

In [ ]:
# Main execution flow
def main():
    try:
        data_raw = pd.read_pickle('raw_data.pkl')
        # Initialize preprocessor and embedder
        preprocessor = RecipePreprocessor(logger = logger)
        # Preprocess data
        df = preprocessor.preprocess_dataframe(data_raw)
        
        # Initialize embedder with state-of-the-art model
        embedder = RecipeEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2",logger = logger)
        
        ingredient_embeddings = embedder.create_ingredient_embeddings(df['ingredients_structured'])
        df['ingredients_embedding'] = ingredient_embeddings
        logger.info(f"Shape of first ingredient embedding: {np.array(ingredient_embeddings[0]).shape}")
        
        
        logger.info("Creating nutrient embeddings...")
        # Create enhanced nutrition embeddings
        nutrition_embeddings = embedder.create_nutrition_embeddings(df['nutrition_vector'].tolist())
        df['enhanced_nutrition_embedding'] = nutrition_embeddings
        logger.info(f"Shape of first nutrition embedding: {np.array(nutrition_embeddings[0]).shape}")
        
        
        # Create text embeddings
        text_embeddings = embedder.create_text_embeddings(df['text_for_embedding'].tolist())
        df['title_instruction_embedding'] = [emb.tolist() for emb in text_embeddings]
        logger.info(f"Shape of first text embedding: {np.array(text_embeddings[0]).shape}")
        
        
        # Create enhanced FSA embeddings
        fsa_embeddings = embedder.create_fsa_embeddings(df['fsa_vector'].tolist())
        df['enhanced_fsa_embedding'] = fsa_embeddings
        logger.info(f"Shape of first FSA embedding: {np.array(fsa_embeddings[0]).shape}")
        
      
        logger.info("Creating full combined embeddings …")
        
        
        df['full_embedding'] = df.apply(create_full_embedding, axis=1)

        # ---------- FAISS index ----------
        full_embeddings = df['full_embedding'].tolist()
        ingredient_embeddings = df['ingredients_embedding'].tolist()


        create_faiss_index(full_embeddings, name = 'full_embeddings')   
        create_faiss_index(ingredient_embeddings,name = 'ingredients_embeddings')  # will build index with d = 1 176
        
        # ---------- save ----------
        df.to_pickle('full_recipe_embeddings.pkl')
        logger.info("Processing & embedding complete")
        return df
        
    except Exception as e:
        logger.error(f"Error during processing: {str(e)}", exc_info=True)

def create_faiss_index(embeddings, name):
    """Create a FAISS index for cosine similarity search"""
    # Convert to numpy array
    embeddings_array = np.array(embeddings).astype('float32')
    
    # Normalize embeddings to unit vectors (for cosine similarity)
    norms = np.linalg.norm(embeddings_array, axis=1, keepdims=True)
    embeddings_array = embeddings_array / np.clip(norms, a_min=1e-8, a_max=None)

    # Get dimensionality
    d = embeddings_array.shape[1]
    
    # Create index with Inner Product (equivalent to cosine when normalized)
    index = faiss.IndexFlatIP(d)
    index.add(embeddings_array)

    # Save index
    faiss.write_index(index, f"recipe_faiss_index_{name}.idx")
    logger.info(f"FAISS index saved to recipe_faiss_index_{name}.idx using cosine similarity")
    
    return index

def create_full_embedding(row):
            return np.concatenate([
                np.array(row['title_instruction_embedding']),   # 384
                np.array(row['enhanced_nutrition_embedding']),  # 15
                np.array(row['enhanced_fsa_embedding']),        # 9
                np.array(row['ingredients_embedding'])          # 384
            ]).tolist()                                         # 


# Integration function to use later with teammate's work
def integrate_with_ingredient_embeddings(your_df, teammate_df):
    """
    Integrate your embeddings with your teammate's ingredient embeddings
    
    Parameters:
    - your_df: Your DataFrame with title, instruction, nutrition, FSA embeddings
    - teammate_df: Your teammate's DataFrame with ingredient embeddings
    
    Returns:
    - Combined DataFrame with all embeddings
    """
    logger.info("Integrating with ingredient embeddings...")
    
    # Create new DataFrame with combined embeddings
    combined_df = your_df.copy()
    
    # Add ingredient embeddings from teammate
    combined_df['ingredients_embedding'] = teammate_df['ingredients_embedding']
    
    # Now create the fully combined embedding
    def create_full_embedding(row):
        # Get all embeddings
        text_emb = np.array(row['title_instruction_embedding'])
        nutrition_emb = np.array(row['enhanced_nutrition_embedding'])
        fsa_emb = np.array(row['enhanced_fsa_embedding'])
        ingredient_emb = np.array(row['ingredients_embedding'])
        
        # Concatenate all embeddings
        combined = np.concatenate([
            text_emb,          # Text embeddings from title and instructions
            nutrition_emb,     # Enhanced nutrition values
            fsa_emb,           # Enhanced FSA values
            ingredient_emb     # Ingredient embeddings from teammate
        ])
        
        return combined.tolist()
    
    # Generate full combined embeddings
    logger.info("Creating full combined embeddings...")
    combined_df['full_embedding'] = combined_df.apply(create_full_embedding, axis=1)
    
    # Create FAISS index for the full embeddings
    full_embeddings = combined_df['full_embedding'].tolist()
    
    create_faiss_index(full_embeddings)
    
    # Save combined data
    combined_df.to_pickle('full_recipe_embeddings.pkl')
    logger.info("Full embeddings saved to full_recipe_embeddings.pkl")
    
    return combined_df

In [4]:
main()

2025-04-30 00:54:55,906 - INFO - Starting data preprocessing...
2025-04-30 00:54:55,907 - INFO - Cleaning text data...


Initializing RecipePreprocessor
RecipePreprocessor initialized


2025-04-30 00:54:57,307 - INFO - Parsing nutrition data...
2025-04-30 00:54:57,308 - INFO - Parsing FSA traffic light data...
2025-04-30 00:54:57,463 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2025-04-30 01:04:13,039 - INFO - Shape of first ingredient embedding: (384,)
2025-04-30 01:04:13,040 - INFO - Creating nutrient embeddings...
2025-04-30 01:04:13,041 - INFO - Creating enhanced nutrition embeddings...
2025-04-30 01:04:13,432 - INFO - Using new scaler...
2025-04-30 01:04:13,444 - INFO - Saving new scaler...
2025-04-30 01:04:13,466 - INFO - Shape of first nutrition embedding: (15,)
Batches: 100%|██████████| 1602/1602 [05:31<00:00,  4.83it/s]
2025-04-30 01:09:46,554 - INFO - Shape of first text embedding: (384,)
2025-04-30 01:09:46,555 - INFO - Creating enhanced FSA embeddings...
2025-04-30 01:09:46,939 - INFO - Shape of first FSA embedding: (9,)
2025-04-30 01:09:46,939 - INFO - Creating full combined embeddings …
2025-04-30 01:09:50,445 - IN

,id,title,ingredients_raw,instructions_full,ingredients_clean,title_nutrition,url_nutrition,instructions,ingredients_structured,nutrition_per_100g,...,fsa_fat,fsa_saturates,fsa_sugars,fsa_salt,fsa_vector,ingredients_embedding,enhanced_nutrition_embedding,title_instruction_embedding,enhanced_fsa_embedding,full_embedding
0,000095fc1d,Yogurt Parfaits,"[8 ounces, weight Light Fat Free Vanilla Yogur...",Layer all ingredients in a serving dish.,"[non - fat vanilla yogurt, strawberries, low -...",Yogurt Parfaits,http://tastykitchen.com/recipes/breakfastbrunc...,Layer all ingredients in a serving dish.,"[{'ingredient': 'yogurt, greek, plain, nonfat'...","{'energy': 81.12946131894766, 'fat': 2.1401392...",...,0,0,1,0,"[0, 0, 1, 0]","[-0.027639709413051605, -0.03064464032649994, ...","[0.09159777890848933, 0.021401392635158917, 0....","[-0.01155478972941637, -0.05617685988545418, 0...","[0.0, 0.0, 1.0, 0.0, 0.25, 3.0, 1.0, 0.0, 0.875]","[-0.01155478972941637, -0.05617685988545418, 0..."
1,00051d5b9d,"Salt Free, Low Cholesterol Sugar Cookies Recipe","[1/2 c. granulated sugar, 3/4 c. salt free cor...",Cream sugar and butter together till smooth. A...,"[granulated sugar, corn oil, Egg Beaters egg s...","Salt Free, Low Cholesterol Sugar Cookies Recipe",http://cookeatshare.com/recipes/salt-free-low-...,Cream sugar and butter together till smooth. A...,"[{'ingredient': 'sugars, granulated', 'quantit...","{'energy': 477.09640393594606, 'fat': 23.41248...",...,2,1,1,1,"[2, 1, 1, 1]","[-0.013630986213684082, -0.024656085297465324,...","[0.5386572302502617, 0.23412485931109794, 0.09...","[-0.03859461471438408, 0.0001314176624873653, ...","[2.0, 1.0, 1.0, 1.0, 1.25, 0.0, 3.0, 1.0, 0.375]","[-0.03859461471438408, 0.0001314176624873653, ..."
2,00059b093b,Honey Sriracha Chicken Wings,"[3 pounds Chicken Wings Or Drummettes, 1 teasp...",Preheat oven to 400 degrees F. In a large bowl...,"[chicken wings, salt, pepper, oil, butter, hot...",Honey Sriracha Chicken Wings,http://tastykitchen.com/recipes/main-courses/h...,Preheat oven to 400 degrees F. In a large bowl...,"[{'ingredient': 'chicken, broilers or fryers, ...","{'energy': 208.05898280960727, 'fat': 14.29704...",...,1,1,0,1,"[1, 1, 0, 1]","[-0.043061405420303345, -0.04862712696194649, ...","[0.23490530317213726, 0.1429704623753859, 0.19...","[-0.011450768448412418, -0.034419894218444824,...","[1.0, 1.0, 0.0, 1.0, 0.75, 1.0, 3.0, 0.0, 0.625]","[-0.011450768448412418, -0.034419894218444824,..."
3,0005fc89f7,Shrimp and Caper Salad,"[2 pounds frozen cooked shrimp without tails, ...","In a large bowl, toss the shrimp, green onions...","[shrimp, green onions, celery, capers, black o...",Shrimp and Caper Salad,http://allrecipes.com/recipe/shrimp-and-caper-...,"In a large bowl, toss the shrimp, green onions...","[{'ingredient': 'crustaceans, shrimp, raw (not...","{'energy': 194.7525956849092, 'fat': 15.980767...",...,1,1,0,1,"[1, 1, 0, 1]","[-0.024970101192593575, -0.048997990787029266,...","[0.2198819628700588, 0.15980767233874782, 0.15...","[-0.09913120418787003, 0.001413632184267044, 0...","[1.0, 1.0, 0.0, 1.0, 0.75, 1.0, 3.0, 0.0, 0.625]","[-0.09913120418787003, 0.001413632184267044, 0..."
4,0006ca31f4,Natural Peanut Butter Chocolate Bon Bons,"[12 cup cocoa powder, 12 cup honey, 1 cup natu...",Measure out the cocoa powder into a mixing bow...,"[cocoa powder, honey, natural - style peanut b...",Natural Peanut Butter Chocolate Bon Bons,http://www.food.com/recipe/natural-peanut-butt...,Measure out the cocoa powder into a mixing bow...,"[{'ingredient': 'cocoa, dry powder, unsweetene...","{'energy': 303.43539955190437, 'fat': 5.094846...",...,1,1,2,0,"[1, 1, 2, 0]","[-0.04592154175043106, -0.03870027884840965, 0...","[0.3425883543327953, 0.050948469006721425, 0.0...","[-0.02296869456768036, 0.007631730753928423, 0...","[1.0, 1.0, 2.0, 0.0, 1.0, 1.0, 2.0, 1.0, 0.5]","[-0.02296869456768036, 0.007631730753928423, 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51230

In [ ]:
user_request = {
    "dietary_preference": 
            "quick, savoury week-night dinner that is high protein, low carb, dairy-free and ready in under 30 minutes,fitness dinner",
            
    # ❶  — INGREDIENTS  ➜ 385-dim ingredient_embedding
    "ingredients": [
        "skinless chicken breast", 
        "broccoli florets", 
        "brown basmati rice", 
    ],
    }


In [7]:
user_input= ", ".join(user_request["ingredients"])

top_k= 50
faiss_index = faiss.read_index('recipe_faiss_index_ingredients_embeddings.idx')
combined_df = pd.read_pickle('full_recipe_embeddings.pkl')


embedder = RecipeEmbedder(model_name="sentence-transformers/all-MiniLM-L6-v2",logger = logger)
        
query_vector = embedder.create_text_embeddings([user_input])

logger.info(f"Shape of first ingredient embedding: {np.array(user_input[0]).shape}")




"""Retrieve initial candidates using FAISS"""
# Perform similarity search
query_vector = query_vector / np.linalg.norm(query_vector)
D, I = faiss_index.search(query_vector.reshape(1, -1).astype('float32'), top_k)

# Retrieve matching recipes
results = combined_df.iloc[I[0]].copy()
results["similarity_score"] = D[0]
results

2025-04-30 01:10:40,435 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 50.18it/s]
2025-04-30 01:10:41,170 - INFO - Shape of first ingredient embedding: ()


,id,title,ingredients_raw,instructions_full,ingredients_clean,title_nutrition,url_nutrition,instructions,ingredients_structured,nutrition_per_100g,...,fsa_saturates,fsa_sugars,fsa_salt,fsa_vector,ingredients_embedding,enhanced_nutrition_embedding,title_instruction_embedding,enhanced_fsa_embedding,full_embedding,similarity_score
10864,361925b75d,Nutty Nuggets,"[1 lb. boneless skinless chicken breasts, cut ...",Preheat oven to 350 degrees F. Place chicken i...,"[boneless skinless chicken breasts, honey must...",Nutty Nuggets,http://www.kraftrecipes.com/recipes/nutty-nugg...,Preheat oven to 350 degrees F. Place chicken i...,"[{'ingredient': 'chicken, broiler or fryers, b...","{'energy': 354.30474264950493, 'fat': 25.31374...",...,1,2,0,"[2, 1, 2, 0]","[-0.014455596916377544, -0.03661929816007614, ...","[0.40002148363653783, 0.25313745291583767, 0.1...","[-0.027348214760422707, -0.016283489763736725,...","[2.0, 1.0, 2.0, 0.0, 1.25, 1.0, 1.0, 2.0, 0.375]","[-0.027348214760422707, -0.016283489763736725,...",0.682342
31069,9b130501e0,Simple Baked Wild Rice,"[1 cup wild rice, rinsed thoroughly and draine...",Put ingredients into 2+ qt covered baking dish...,"[wild rice, warm water, instant bouillon granu...",Simple Baked Wild Rice,http://www.food.com/recipe/simple-baked-wild-r...,Put ingredients into 2+ qt covered baking dish...,"[{'ingredient': 'wild rice, raw', 'quantity': ...","{'energy': 66.86444545869466, 'fat': 0.2932450...",...,0,0,1,"[0, 0, 0, 1]","[-0.04366404563188553, -0.020304853096604347, ...","[0.0754921158404617, 0.0029324509356458417, 0....","[0.051303088665008545, 0.026374027132987976, 0...","[0.0, 0.0, 0.0, 1.0, 0.25, 3.0, 1.0, 0.0, 0.875]","[0.051303088665008545, 0.026374027132987976, 0...",0.681949
9468,2f321a1e56,Grilled Tuscan Chicken,"[2 lbs chicken, breasts halves, boneless, skin...",Place chicken in large resealable plastic bag ...,"[chicken, olive oil]",Grilled Tuscan Chicken,http://www.food.com/recipe/grilled-tuscan-chic...,Place chicken in large resealable plastic bag ...,"[{'ingredient': 'chicken, broiler or fryers, b...","{'energy': 211.96612403688556, 'fat': 2.849543...",...,0,2,0,"[0, 0, 2, 0]","[-0.020166028290987015, -0.023666320368647575,...","[0.23931659165454822, 0.028495434794020363, 0....","[-0.02323191426694393, -0.042785391211509705, ...","[0.0, 0.0, 2.0, 0.0, 0.5, 3.0, 0.0, 1.0, 0.75]","[-0.02323191426694393, -0.042785391211509705, ...",0.676153
20238,65e5e680b4,Simple Honey Mustard Chicken,"[1 1/2 pounds chicken breast halves, boneless,...",Place chicken in a 13 by 9 inch baking dish. M...,"[chicken breast halves, honey, butter, Dijon m...",Simple Honey Mustard Chicken,https://recipeland.com/recipe/v/simple-honey-m...,Place chicken in a 13 by 9 inch baking dish. M...,"[{'ingredient': 'chicken, broiler or fryers, b...","{'energy': 194.5735620153354, 'fat': 5.6835686...",...,1,2,1,"[1, 1, 2, 1]","[-0.024479461833834648, -0.0124303437769413, 0...","[0.2196798280818303, 0.056835686029286375, 0.1...","[-0.0425298847258091, 0.014155863784253597, 0....","[1.0, 1.0, 2.0, 1.0, 1.25, 0.0, 3.0, 1.0, 0.375]","[-0.0425298847258091, 0.014155863784253597, 0....",0.674545
16989,55188e6209,Simple Honey Mustard Chicken,"[1 1/2 pounds chicken breast halves, boneless,...",Place chicken in a 13 by 9 inch baking dish. M...,"[chicken breast halves, honey, butter, Dijon m...",Simple Honey Mustard Chicken,https://recipeland.com/recipe/v/simple-honey-m...,Place chicken in a 13 by 9 inch baking dish. M...,"[{'ingredient': 'chicken, broiler or fryers, b...","{'energy': 194.5735620153354, 'fat': 5.6835686...",...,1,2,1,"[1, 1, 2, 1]","[-0.024479461833834648, -0.0124303437769413, 0...","[0.2196798280818303, 0.056835686029286375, 0.1...","[-0.0425298847258091, 0.014155863784253597, 0....","[1.0, 1.0, 2.0, 1.0, 1.25, 0.0, 3.0, 1.0, 0.375]","[-0.0425298847258091, 0.014155863784253597, 0....",0.674545
41967,d1d80b157b,Baked Brown Sugar Chicken,"[3 -4 lbs boneless skinless chicken, 1 23 ounc...",Preheat oven to 350. Rinse the 